In [29]:
#Here you must specify your own path
root = ''

In [30]:
import numpy as np
import pandas as pd
import sklearn.model_selection as model_selection
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

In [31]:
# Dic to get the respective target values of the numbers.
target_dic = {
    0: '(0, 20]constant',
    1: '(20, 40]constant',
    2: '(40, 60]constant',
    3: '(60, 80]constant',
    4: '(80, 100]constant',
    5: '(0, 20]intermittent',
    6: '(20, 40]intermittent',
    7: '(40, 60]intermittent',
    8: '(60, 80]intermittent',
    9: '(80, 100]intermittent',
}

In [32]:
#read in csv file into dataframe
df_train = pd.read_csv(root + 'data/02_processed/train.csv', sep=';')
df_train.pop('Unnamed: 0')

df2=df_train.copy()
df2.drop(['target','4','66'], axis = 1,inplace=True)


#read validation and test data from csv
df_validation = pd.read_csv(root + 'data/02_processed/validation.csv', sep=';')
df_validation.set_index('user_id',inplace=True)
df_test = pd.read_csv(root + 'data/02_processed/test.csv', sep=';')
df_test.set_index('user_id',inplace=True)

#encode target column of test data
target_test=pd.get_dummies(df_test['target'], prefix='target')

# target of train data without encoding
target_train= df_train['target']

#delete target from training data
df_train.pop('target')

# target of train data without encoding
target_test= df_test['target']

#delete target from training data
df_test.pop('target')

train=df_train.copy()
test= df_test.copy()

In [33]:
#Get the categorical and numerical column names
categorical_columns=[]
numerical_columns=[]
for name, values in train.iteritems():
    if values.dtype==np.float64:
        numerical_columns.append(name)
    if values.dtype==object:
        categorical_columns.append(name)

numerical_columns  

['4', '66', '67']

In [34]:
#OneHotEncoder
categorical_encoder = OneHotEncoder(handle_unknown='ignore')

categorical_encoder2 = OrdinalEncoder()

#ColumnTransformer encodes categorical data with the categorical_encoder.
#and numeric columns are not edited.
preprocessing = ColumnTransformer(
    [('cat', categorical_encoder2, categorical_columns),
     ('num', 'passthrough', numerical_columns)])

scorer = make_scorer(f1_score,average='macro')

Dtc = DecisionTreeClassifier(random_state=0)

# This grid contains values which are tested by the Randomized search 
#to get the best Combinations of them, measured with the  f1-score
param_grid = [
    {'max_depth':[4,6,8,10,15,20,25,30],
     'min_samples_split': [2,4,8,12,16,32,40],
     'min_samples_leaf':[2,4,6,8,10,20],
     'max_features':[8,10,11,20],
    }
]

#Created Pipeline to combine preprocessing of the data and the gridsearch.
model = Pipeline([
    ('preprocess', preprocessing),
    ('gridsearch',GridSearchCV(Dtc,param_grid,cv=5,scoring=scorer,n_jobs=-1,verbose=3,return_train_score=True))
])

model.fit(train, target_train)
model.named_steps['gridsearch'].best_params_

Fitting 5 folds for each of 1344 candidates, totalling 6720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 1880 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 3672 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 5976 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 6720 out of 6720 | elapsed:   43.9s finished


{'max_depth': 25,
 'max_features': 20,
 'min_samples_leaf': 2,
 'min_samples_split': 2}

In [35]:
model.named_steps['gridsearch'].best_score_

0.7909413450425457